In [163]:
import pandas as pd
import numpy as np
def main():
    df_comp = pd.read_csv ("../company.csv",encoding='utf8',sep=";")
    df_comp = pd.DataFrame(df_comp,columns=["Raison sociale","Adresse normée ligne 4","Ville","Code postal",
                                      "Libellé activité","Code activité","Date de création","Nom dirigeant principal","Prénom dirigeant principal",
                                      "Tranche Effectif INSEE","Tranche Effectif établissement","Téléphone","Email","Chiffre d'affaires"])
    print(df_comp["Email"])
    
    df_cont = pd.read_csv ("../contact.csv",encoding='utf8',sep=";")
    df_cont = pd.DataFrame(df_cont,columns=[
        "Civilité","Nom","Prénom","Fonction","Raison sociale","Téléphone","Email"
    ])
    # print(df_cont[df_comp["Nom"].duplicated()==True])
        
    # print(df_cont)
    return df_comp,df_cont
df_comp,df_cont = main()

0        contact@victorhugohotel.com
1                                NaN
2                                NaN
3                                NaN
4           contact@pmc-decoupage.fr
                    ...             
29797                            NaN
29798    reception@hotelmagellan.com
29799                            NaN
29800                            NaN
29801                            NaN
Name: Email, Length: 29802, dtype: object


In [164]:
import re
import math

def is_nan(x):
    return math.isnan(x)

def format_phone(phone):
    if(not is_nan(phone)):
        phone = int(phone)
        phone = '0'+str(phone)
        # Enlève tous les caractères qui ne sont pas des chiffres
        prefixe = '+33'
        numero = phone.replace(' ', '')
        numero = numero.replace('-', '')
        numero = numero.replace('.', '')
        numero = prefixe + numero[1:]
        numero_formatte = numero[0:3] + ' ' + numero[3]+' '+' '.join([numero[i:i+2] for i in range(4, len(numero), 2)])
        return numero_formatte
    else:
        return phone

In [165]:
df_comp['Téléphone']

0        145531491.0
1                NaN
2        147952695.0
3        153338200.0
4        164255087.0
            ...     
29797            NaN
29798    145724451.0
29799    148265706.0
29800    146266030.0
29801    169015447.0
Name: Téléphone, Length: 29802, dtype: float64

In [166]:
df_cont['Téléphone'] = df_cont['Téléphone'].apply(format_phone)
df_comp['Téléphone'] = df_comp['Téléphone'].apply(format_phone)

In [167]:
df_cont['Téléphone']

0        +33 1 45 53 14 91
1        +33 1 45 53 14 91
2        +33 1 45 53 14 91
3        +33 1 45 53 14 91
4        +33 1 45 53 14 91
               ...        
32411    +33 1 45 72 44 51
32412    +33 1 45 72 44 51
32413    +33 1 48 26 57 06
32414    +33 1 46 26 60 30
32415    +33 1 69 01 54 47
Name: Téléphone, Length: 32416, dtype: object

In [168]:
for index,z in df_cont[df_cont['Raison sociale']=="SOCIETE HOTELIERE PARIS-OUEST"].iterrows():
    print(z)

Civilité                                     M.
Nom                                       STOLL
Prénom                                    Alain
Fonction                    Directeur Financier
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 1 45 72 44 51
Email               reception@hotelmagellan.com
Name: 32409, dtype: object
Civilité                                    MME
Nom                                    DEQUAIRE
Prénom                                   Hélène
Fonction                   Directeur des Achats
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 1 45 72 44 51
Email               reception@hotelmagellan.com
Name: 32410, dtype: object
Civilité                                    MME
Nom                                    DEQUAIRE
Prénom                                   Hélène
Fonction           Dir. des Ressources Humaines
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 

In [186]:
#TEST API
import requests
import json
import pprint


# Définissez l'URL de l'API d'importation
df_comp = df_comp.fillna("lol")
df_cont = df_cont.fillna("lol")
entreprises=[]
for index,x in df_comp.head(10).iterrows():
      my_company = {
        "type": "company",
        "properties": {
          "name": {"value": x['Raison sociale']},
          "Type principal":{"value" : x['Libellé activité']},
          "Téléphone":{"value":str(x['Téléphone'])},
          "Email":{"value":x['Email']}
        }
      }
      entreprises.append(my_company)
      for index,z in df_cont.head(10)[df_cont['Raison sociale']==x['Raison sociale']].iterrows():
            my_contact = {
              "type": "contact",
              "properties": {
                "Prenom":{"value":z['Prénom']},
                "Nom":{"value":z['Nom']},
                "Téléphone":{"value":str(z['Téléphone'])},
                "company":{"value":z['Raison sociale']}
                }
            }
            entreprises.append(my_contact) 

# Créez le corps de la requête
data = {
  "importId": "my_import",
  "createIfNotExists": "true",
  "inputs": entreprises
}
# pprint.pprint(data)
data = json.dumps(data,ensure_ascii=False)
pprint.pprint(data)
print(data)

('{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": '
 '"company", "properties": {"name": {"value": "SOCIETE DE PARTICIPATION ET '
 'D\'EXPLOITATION HOTELIERE SOPAROTEL"}, "Type principal": {"value": "Hôtels '
 'et hébergement similaire"}, "Téléphone": {"value": "+33 1 45 53 14 91"}, '
 '"Email": {"value": "contact@victorhugohotel.com"}}}, {"type": "contact", '
 '"properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": '
 '"QUENNESSON"}, "Téléphone": {"value": "+33 1 45 53 14 91"}, "company": '
 '{"value": "SOCIETE DE PARTICIPATION ET D\'EXPLOITATION HOTELIERE '
 'SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": '
 '"Olivier"}, "Nom": {"value": "QUENNESSON"}, "Téléphone": {"value": "+33 1 45 '
 '53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET '
 'D\'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": '
 '{"Prenom": {"value": "Maguelone"}, "Nom": {"value": "ARTIERES"}, '
 '"Téléphone": {"value": "+33 

c:\Users\Icet0\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [202]:
data = {
  "importId": "my_import",
  "createIfNotExists": "true",
  "inputs": [
    {
      "type": "company",
      "properties": {
        "name": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" },
        "Type principal": { "value": "Hôtels et hébergement similaire" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "Email": { "value": "contact@victorhugohotel.com" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Olivier" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Olivier" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
        {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Maguelone" },
        "Nom": { "value": "ARTIERES" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Maguelone" },
        "Nom": { "value": "ARTIERES" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Jacques" },
        "Nom": { "value": "Blanc" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Marie-Hélène" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "company",
      "properties": {
        "name": { "value": "ETABLISSEMENTS TIANO" },
        "Type principal": { "value": "Intermédiaires du commerce en machines, équipements industriels, navires et avions" },
        "Téléphone": { "value": "de" },
        "Email": { "value": "de" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "ALFRED JACQUES" },
        "Nom": { "value": "TIANO" },
        "Téléphone": { "value": "de" },
        "company": { "value": "ETABLISSEMENTS TIANO" }
      }
    },
  ]}
# transforme le dictionnaire en json
data_string = json.dumps(data)
payload = {"importRequest": data}
print(payload["importRequest"])
# Envoyez la requête POST à l'API d'importation
# Définissez votre clé API d'HubSpot
api_key = "pat-eu1-91242dbb-e9d6-4b77-852f-fdd31151e049"
url = "https://api.hubapi.com/crm/v3/imports"
headers = {
  # 'content-type': 'multipart/form-data',
  # 'accept': 'application/json',
  'Authorization': 'Bearer %s' % api_key
}


response = requests.request("POST", url, data=payload,headers=headers)
print(response.encoding)
print(response.text.encode('utf8'))
print(response.status_code)
# Affichez le statut de la réponse
print(response.status_code)

# Si la réponse est un succès (code 200), affichez le message de réussite
if response.status_code == 200:
    print('Import réussi !')
# Sinon, affichez le message d'erreur
else:  
    print('Erreur lors de l\'import !')
    print(response.json())


{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": "company", "properties": {"name": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}, "Type principal": {"value": "H\u00f4tels et h\u00e9bergement similaire"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "Email": {"value": "contact@victorhugohotel.com"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": "QUENNESSON"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": "QUENNESSON"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Maguelone"}, "Nom": {"value": "ARTIERES"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [205]:
import requests

# Définissez votre jeton d'API et l'URL de l'API d'import
api_key = "pat-eu1-91242dbb-e9d6-4b77-852f-fdd31151e049"
import_url = "https://api.hubapi.com/crm/v3/imports"

# Préparez le corps de la requête au format JSON
import_data = {
    "importId": "my_import",
    "createIfNotExists": "true",
    "inputs": [
        {
            "type": "company",
            "properties": {
                "name": { "value": "Acme Corp" },
                "phone": { "value": "+1 555 555 5555" }
            }
        },
        {
            "type": "contact",
            "properties": {
                "firstName": { "value": "John" },
                "lastName": { "value": "Doe" },
                "phone": { "value": "+1 555 555 5555" },
                "company": { "value": "Acme Corp" }
            }
        }
    ]
}

data = json.dumps(import_data)
# Envoyez la requête HTTP POST à l'API d'import
headers = { "Authorization": f"Bearer {api_key}" }
response = requests.post(import_url, json=data, headers=headers)

# Affichez le statut de la réponse et le corps de la réponse (s'il y en a un)
print(response.status_code)
print(response.text)


{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": "company", "properties": {"name": {"value": "Acme Corp"}, "phone": {"value": "+1 555 555 5555"}}}, {"type": "contact", "properties": {"firstName": {"value": "John"}, "lastName": {"value": "Doe"}, "phone": {"value": "+1 555 555 5555"}, "company": {"value": "Acme Corp"}}}]}
415
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
<title>Error 415 Unsupported Media Type</title>
</head>
<body><h2>HTTP ERROR 415</h2>
<p>Reason:
<pre>    Unsupported Media Type</pre></p>
</body>
</html>

